In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

from pim.models.network2 import RecurrentNetwork, Layer, InputLayer, Connection, WeightedConnection
from pim.models.new.stone import tb1_output, tn1_output, tn2_output, cpu4_output, cpu1_output, motor_output, CentralComplex

In [ ]:
def linear_neuron(gain, gain_noise, noise):
    gain = np.random.normal(gain, gain_noise)
    def f(x):
        return gain * x + np.random.normal(0, noise)
    return f

def bistable_neuron(Idown, Iup, gain, gain_noise, noise):
    gain = np.random.normal(gain, gain_noise)
    state = 0
    def f(x):
        nonlocal state
        if x >= Iup:
            state = 1
        elif x <= Idown:
            state = 0            
        return state * gain * x + np.random.normal(0, noise)
    return f

In [ ]:
class BistableLayer(Layer):
    def __init__(self, name, N, dI, mI, gain, gain_noise, noise):
        super().__init__(name)
        self.N = N
        self.mI = mI
        self.gain = np.random.normal(gain, gain_noise)
        self.noise = noise
        print([I_down for I_down in np.linspace((mI-dI)/N, mI, N)])
        self.neurons = [bistable_neuron(I_up-dI, I_up, 1.0, 0.0, noise) for I_up in np.linspace((mI-dI)/N, mI, N)]
    
    def update(self, x):
        activity = np.array([neuron(x) for neuron in self.neurons])
        num_active = np.sum(activity > 0.5)
        return num_active / self.N * self.mI * self.gain + np.random.normal(0.0, self.noise)

class LinearLayer(Layer):
    def __init__(self, name, gain, gain_noise, noise):
        super().__init__(name)
        self.neuron = linear_neuron(gain, gain_noise, noise)
    
    def update(self, x):
        return self.neuron(x)
    
def step(network, input_layer, memory, i):
    input_layer.set(i)
    network.step()
    return memory.output


In [ ]:
T = np.linspace(0, 20, 100)
I = np.zeros(T.size)
I[(0.2 < T) & (T < 0.3)] = 1.0
I[(2.5 < T) & (T < 3.5)] = 1.0
I[(7.5 < T) & (T < 8)] = -1.0
I[(12 < T) & (T < 15)] = (T[(12 < T) & (T < 15)]-12)*0.05

plt.figure()
plt.plot(T, I)
plt.plot(T, np.cumsum(I))
plt.show()

In [ ]:
network = RecurrentNetwork()

input_layer = network.add_layer(InputLayer("in"))
memory = network.add_layer(BistableLayer("mem", 40, 0.2, 10, 1.03, 0.00, 0.02))
#memory = network.add_layer(LinearLayer("mem", 1.0, 0.00, 0.05))

network.add_connection(WeightedConnection("in", "mem", 1.0))
network.add_connection(WeightedConnection("mem", "mem"))

output = np.array([step(network, input_layer, memory, i) for i in I])

plt.figure()
plt.plot(T, I, label="input")
plt.plot(T, np.cumsum(I), label="true integral")
plt.plot(T, output, label="memory")
plt.legend()